# LangChain Information Gathering Conversational System

This notebook demonstrates an AI-assisted information gathering conversation system. The system uses the Anthropic Claude model to engage in a natural conversation with the user, asking for specific pieces of information one at a time. The gathered information is stored in a structured format using a Python dictionary.

## Setting Up
Uncomment to install the package

In [158]:
# pip install -U langchain-anthropic

Uncomment if API key is not added yet

In [159]:
# import getpass
# import os

# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

## The basic: Converting Responses to Structured Output
This section covers the basics of how to effectively capture information from customers and convert their responses into a structured, machine-readable format.

In [160]:
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field
from typing import Optional

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

class PersonalDetails(BaseModel):
    first_name: Optional[str] = Field(description="This is the first name of the user.")
    last_name: Optional[str] = Field(description="This is the last name of the user.")
    city: Optional[str] = Field(
        description="The name of the city where the user lives",
    )
    email: Optional[str] = Field(
        description="an email address that the person associates as the user",
    )

llm_with_structured_output = model.with_structured_output(PersonalDetails)

user_input = "Hi my name is John Doe, and I live in New York."
response = llm_with_structured_output.invoke(user_input)
print(response)

first_name='John' last_name='Doe' city='New York' email='<UNKNOWN>'


## Building Information Gathering Conversation System
The system is designed to interact with customers in a natural, conversational manner to collect specific pieces of information. The gathered information is stored in a Python dictionary called `form_result`.

The assistant engages in a back-and-forth dialog with the user, asking for each required piece of information one at a time and waiting for the user's response before proceeding to the next item.

In [161]:
customer_details = PersonalDetails(first_name="", last_name="", city="", email="")
print(customer_details)

first_name='' last_name='' city='' email=''


In [162]:
from langchain_core.prompts import ChatPromptTemplate

form_result = {
    "first_name": "",
    "last_name": "",
    "city": "",
    "email": ""
}

def gather_info(form_result):
    info_gathering_prompt_template = ChatPromptTemplate.from_messages([
        ('system', """<context>
    You are an AI assistant having a conversation with the user to gather some required information. You should ask for the information in a natural, conversational way.
    </context>

    <instructions>
    - Do not greet the user, say hi, or introduce yourself because it is assumed that the user is already aware that they are interacting with an AI assistant
    - Ask for each piece of information one at a time, waiting for the user's response before moving on to the next item
    - Don't list out all the required information upfront
    - Phrase your requests as questions that flow naturally in the conversation 
    - If the user provides information you didn't explicitly ask for yet, acknowledge it and don't ask for it again later
    - Once you have collected all the required information or if there is no information to collect (the ask_for list is empty), simply say thank you and tell the user that they are being connected to a human agent and end the conversation.
    </instructions>"""),
        ('user', """<ask_for>
    {ask_for}
    </ask_for>""")
    ])
    info_gathering_llm = info_gathering_prompt_template | model
    empty_string_keys = [key for key, value in form_result.items() if value == "" or value == "<UNKNOWN>"]
    response_content = info_gathering_llm.invoke({"ask_for": empty_string_keys}).content
    return response_content

In [163]:
def update_dict_if_not_empty(target_dict, update_dict):
    for key, value in update_dict.items():
        if target_dict.get(key) in ("", "<UNKNOWN>") and value not in ("", "<UNKNOWN>"):
            target_dict[key] = value
    return target_dict

def update_form_with_user_input(form_result, user_input):
    response = llm_with_structured_output.invoke(user_input)
    print("New data from latest user input: ", response.dict())
    form_result = update_dict_if_not_empty(form_result, response.dict())
    return form_result

In [172]:
# Start the conversation with any predefined message
print("Hello! I'm an AI assistant and I'm here to help you today. To provide the best assistance, there are a few key pieces of information I need to collect from you first, if that's alright. So to start off, could you please answer the following questions?")

Hello! I'm an AI assistant and I'm here to help you today. To provide the best assistance, there are a few key pieces of information I need to collect from you first, if that's alright. So to start off, could you please answer the following questions?


In [164]:
response_content = gather_info(form_result)
print(response_content)

What's your first name?


In [165]:
user_input = "Hey I'm John Doe"
form_result = update_form_with_user_input(form_result, user_input)
print(form_result)

New data from latest user input:  {'first_name': 'John', 'last_name': 'Doe', 'city': '<UNKNOWN>', 'email': '<UNKNOWN>'}
{'first_name': 'John', 'last_name': 'Doe', 'city': '', 'email': ''}


In [166]:
response_content = gather_info(form_result)
print(response_content)

So, where are you located? What city do you call home?


In [167]:
user_input = "I currently live in New York."
form_result = update_form_with_user_input(form_result, user_input)
print(form_result)

New data from latest user input:  {'first_name': '<UNKNOWN>', 'last_name': '<UNKNOWN>', 'city': 'New York', 'email': '<UNKNOWN>'}
{'first_name': 'John', 'last_name': 'Doe', 'city': 'New York', 'email': ''}


In [168]:
response_content = gather_info(form_result)
print(response_content)

So, what email address would be best for us to reach you at?


In [169]:
user_input = "My email is johndoe@example.com , and my cousin's email is janedoe@example.com just in case you need it."
form_result = update_form_with_user_input(form_result, user_input)
print(form_result)

New data from latest user input:  {'first_name': '<UNKNOWN>', 'last_name': '<UNKNOWN>', 'city': '<UNKNOWN>', 'email': 'johndoe@example.com'}
{'first_name': 'John', 'last_name': 'Doe', 'city': 'New York', 'email': 'johndoe@example.com'}


In [170]:
response_content = gather_info(form_result)
print(response_content)

Thank you for providing that information. You are now being connected to a human agent. Have a great day!
